In [34]:
## History data of stocks
## Recent News
## Ratios like P/E, P/B, ROE, EPS, Industry P/E   
# Analyse all the data and recommend the stock based on the ReAct

## Stock Bot

In [ ]:
# !pip install feedparser --quiet
# !pip install streamlit  --quiet
# !pip install jupyter_contrib_nbextensions

In [4]:
import os
import re
import requests
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import pandas as pd
import json
from googlesearch import search
import yfinance as yf
import feedparser
from urllib.parse import quote_plus

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks import get_openai_callback
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType


In [37]:
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key')

Enter OpenAI API key ········


In [7]:
nse_data =  pd.read_excel('NSE_Symbols.xlsx')
nse_data.drop(columns= [ 'Sr. No.','Market capitalization as on March 28, 2024\n(In lakhs)'],inplace =True)

### Fetching Company Symobl

In [8]:
def find_company_symbol(company_name, nse_data):
    
    company_name = company_name + " LTD"
    json_data = nse_data.to_json(orient='split')
    json_data = json.loads(json_data)
    data = json_data['data']
    
    # Initialize the LangChain ChatOpenAI model
    # Initialize the ChatOpenAI model
    model = ChatOpenAI(model_name="gpt-4o-mini")


    # Define the function schema for symbol extraction
    function_symbol = [{
        "name": "function_symbol",
        "description": "Find symbol for company from given JSON data. If a match exists, return the symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The symbol for the company if found in the JSON data."
                },
                "company_name": {
                    "type": "string",
                    "description": "The name of the company given in JSON data."
                },
            },
            "required": ["symbol", "company_name"],
        },
    }]

    system_msg = SystemMessage(content="""
            You are an analyst that finds the symbol of a company from given JSON data. Each item in the JSON object contains the symbol of a company and its name.
            For a given company name, you will look for each name in the given JSON object, then find the one which looks most similar to the given name
            and return the corresponding symbol and name that you found in JSON data.
            """
             )

    human_msg = HumanMessage(content=  f"""
            Find the symbol and name of {company_name}.

            JSON data is:
            {data}
            """)
   
    # Call the model with function calling using LangChain's updated method
   
    with get_openai_callback() as cb:
        response = model.invoke(
            [system_msg, human_msg],
            functions=function_symbol,
            function_call={"name": "function_symbol"}
        )

    function_call = response.additional_kwargs['function_call']
    company_info = json.loads(function_call['arguments'])
    
    
    return {'symbol': company_info['symbol'], 'full_name':company_info['company_name'].replace('&','').lower().replace(' ','-')}

In [14]:
company_name =  "adani green"
find_company_symbol(company_name, nse_data)


{'symbol': 'ADANIGREEN', 'full_name': 'adani-green-energy-ltd'}

### Meeting Yahoo

In [9]:
''' Fetch stock data from Yahoo Finance '''

def get_stock_price(ticker,history =10):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="5d")  #['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
    df=df[["Close","Open", "Volume"]] 
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df.reset_index(inplace = True)
    df=df[-history:]

    json_data = df.to_json(orient='split')
    data = json.loads(json_data)['data']
    
    return {'past_5_days_data': data}


In [10]:
ticker = "RELIANCE"
get_stock_price(ticker)

{'past_5_days_data': [['2024-12-16', 1268.3000488281, 1275.0, 9486781],
  ['2024-12-17', 1245.3000488281, 1261.0500488281, 17462791],
  ['2024-12-18', 1253.25, 1240.6500244141, 12670179],
  ['2024-12-19', 1230.4499511719, 1239.0, 14244653],
  ['2024-12-20', 1205.3000488281, 1224.0, 20312896]]}

### News Headlines

In [11]:
def get_google_news_headlines(company_name, max_limit= 10):
    # Google News RSS feed URL for the given company
    rss_url = f"https://news.google.com/rss/search?q={quote_plus(company_name)}&hl=en-US&gl=US&ceid=US:en"
    
    # Parse the RSS feed
    news_feed = feedparser.parse(rss_url)
    
    # Extract the top 5 headlines
    headlines = []
    for entry in news_feed.entries[:max_limit]:  # Get only the top 5 news entries
        headlines.append(entry.title)
        
    return {'recent_news':headlines}


In [12]:
company ="Hindusten Unilver"
top_headlines = get_google_news_headlines(f'{company}+company')
top_headlines

{'recent_news': ["India's Hindustan Unilever reports higher Q1 profit on volume growth - Reuters.com",
  'Hindustan Unilever Wins Digital Supply Chain at P&SC Awards - Supply Chain Digital',
  'Hindustan Unilever sees 10 of its brands at the sweet spot to go premium - Business Standard',
  'Risks To Shareholder Returns Are Elevated At These Prices For Hindustan Unilever Limited (NSE:HINDUNILVR) - Simply Wall St',
  'Hindustan Unilever dips into luxury retail with Hourglass launch - Inside Retail Asia',
  'Hindustan Unilever to reduce palm oil content in soaps by 25% - Oils & Fats International',
  'Hindustan Unilever Limited Board approves demerger of Ice Cream business into an independent listed entity - The Hindu',
  'Hindustan Unilever Q2 Results Live: Profit Falls by 2.45% YOY | Company Business News - Mint',
  'Hindustan Unilever shares tumble nearly 6% post earnings; market valuation falls by Rs 36,430 crore - Deccan Herald',
  'Hindustan Unilever Limited to separate its ice crea

### Fetching Financial Statements

### Important Ratios

In [32]:
def get_company_fundamentals(company_name):

    url = f"https://groww.in/stocks/{company_name}"
    # Send a request to fetch the HTML content of the page with a timeout
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'},timeout =5)
    soup = BeautifulSoup(response.content, 'lxml')  # Using 'lxml' for faster parsing
    # Find the specific div container and then the table within it
    container = soup.find('div', class_='row l12 ft785TableContainer')
    rows = []
    
    if container:
        table = container.find('table', class_='tb10Table col l12 ft785Table')
        print(table)
        if table:
            for row in table.find_all('tr'):
                cells = [cell.text.strip() for cell in row.find_all(['th', 'td'])]
                rows.append(cells)
        else:
            print("Table with the specified class not found within the container.")
    else:
        print("Container with the specified class not found.")
    
    # Print all rows
    dict = {} 
    for row in rows:
        dict[row[0]] = row[1]
    
    # del dict['Market Cap']
    json_data = json.dumps(dict)
    data = json.loads(json_data)
    return {'fundamental_ratios': data}   # Returning data in json format



### Integrating all Tools

In [35]:
def stock_analyzer(company_name, data_df):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
    
    tools = [
        Tool(
            name="find_company_symbol",
            func=lambda inputs: find_company_symbol(company_name, data_df),
            description="Extracts symbol and full name of a company using company name and DataFrame.",
            return_direct=False
    
        ),
        
        Tool(
            name="GetPast5DaysData",
            func=lambda symbol: get_stock_price(symbol.strip('"').strip("'").strip().strip("'").strip('"')) ,
        #     func=lambda symbol: (
        #     print(f"GetPast5DaysData received symbol: {symbol.strip()}"),  # Debugging line
        #     get_stock_price(symbol.strip())
        # )[1],
            description="Fetches past 5 days stock price and volume; requires precise symbol of the company's stock",
            return_direct=False
        ),
        Tool(
            name="GetRecentNews",
            func=lambda inputs: get_google_news_headlines(company_name),
            description="Fetches recent news headlines about company",
            return_direct=False
        ),
        Tool(
            name="GetFunadamentalRatios",
            # func=lambda inputs: get_company_fundamentals(inputs['full_name']) if inputs["full_name"] else None,
            func=lambda full_name: get_company_fundamentals(full_name.strip('"').strip("'").strip().strip("'").strip('"')),
        #     func=lambda symbol: (
        #     print(f"GetPast5DaysData received symbol: {symbol.strip()}"),  # Debugging line
        #     get_company_fundamentals(symbol.strip())
        # )[1],
            description="Gets fundamental ratios and values of a company; requires the precise full name of the company which you might obtain from find_company_symbol.",
            return_direct=False
        ),
    ]
    
    # Main function to handle agent-based analysis
    def analyze_stock_with_agent(company_name, data_df):
        # Provide initial context to agent, guiding it on dependencies
        context = (
            f"Company Name: {company_name}\n"
            "Goal: Extract symbol and full name, then use symbol to fetch past 5 days stock data and news. "
            "Use full name to retrieve fundamental ratios.\n"
            "If certain information is missing (like symbol), extract it first."
            "When completing actions, please respond in this exact format:\n\n"
            "Thought: [Your thought process]\n"
        "Action: [Name of the action]\n"
        "Action Input: [The input required for the action]\n\n"

       "Following are the points that you can consider while analysing :\n"
       "1. If past day prices are monotonic i.e. either increasing or decreasing, then one can say it is a good or bad sign resepctively"
       "2. If past day prices are not montonic and vary from each other, it means stock is volatile and risky at moment"
       "3. Doing sentimental Analysis of recent news can tell us whether there is positive response for company from market or not"
       "4. If P/E Ratio (TTM) is significantly greater than industry P/E ratio and P/B ratio is significantly greater, then this mean stock is overvalued"
       "5. ROE greater than 12% is generally considered as good"
       "6. Lower D/E ratio, usually smaller than 0.4, is considered as good"
       "7. P/B ratio, lower than 10, is usually considered as good"

        "Based on the above all information, please answer the following:\n"
        "1. Is the company in good financial condition?\n"
        "2. Should the stock be bought or sold?\n"
        "3. If buying, suggest an ideal holding period (short-term, medium-term, or long-term).\n"

        "Please Provide proper explanation, reason or rationale behind answers of above question"


    )
  
    # Run agent to gather and analyze data
        response = agent.run(input = context, company_name=company_name, data_df=data_df)
        return response

       # Initialize the agent, which will select tools based on context
    agent = initialize_agent(
        tools=tools,
        llm=llm,  # Your language model
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose= True
    )
    

    company_name = company_name
    data_df = data_df # Replace with actual DataFrame object
    recommendation = analyze_stock_with_agent(company_name, data_df)
    return recommendation


In [36]:
import ipywidgets as widgets
from IPython.display import display

# Define the widgets
company_name_widget = widgets.Text(value='', description='Company Name:', placeholder='Enter the name of the company')
analyze_button = widgets.Button(description='Analyze Stock', button_style='primary')
output_area = widgets.Output()

# Display widgets
display(company_name_widget, analyze_button, output_area)

# Define the function to analyze stock with the agent
def analyze_stock(b):
    output_area.clear_output()  # Clear previous output before new analysis
    with output_area:
        company_name = company_name_widget.value.strip()  # Capture and strip latest company name
        # print(company_name,"hgjg")
        data_df = nse_data  # Replace with actual DataFrame object
    
        try:
            print(f"Analyzing stock for: {company_name}")  # Print the current company being analyzed
            recommendation = stock_analyzer(company_name, data_df)
            print("Summary:", recommendation)
        except Exception as e:
            print(f"An error occurred: {e}")

# Link button to the function
analyze_button.on_click(analyze_stock)


Text(value='', description='Company Name:', placeholder='Enter the name of the company')

Button(button_style='primary', description='Analyze Stock', style=ButtonStyle())

Output()